In [95]:
from time import sleep
from deep_translator import GoogleTranslator
import requests
import re
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import XSD
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, FOAF
from shapely.geometry import Point, Polygon


## Código para preprocesar las propiedades de los features:

In [ ]:


def make_synonimous(word,source_lang,target_lang):
    url = f"https://babelnet.io/v8/getSynsetIds?lemma={word}&searchLang={source_lang}&targetLang	={target_lang}&key=5b0c06e3-6791-4869-ac51-d3f026600721"

    response = requests.get(url)
    synset_ids = response.json()

    resul = None
    translations = []
    for synset_id in synset_ids:
        id = synset_id['id'].split(':')[1]
        url = f"https://babelnet.io/v8/getSynset?id=bn:{id}&key=5b0c06e3-6791-4869-ac51-d3f026600721"
        response = requests.get(url)
        synset_data = response.json()
        for translation in synset_data['senses']:
            resul = translation['properties']['lemma']['lemma']

    return resul

def make_suggestion(word):
    url = "https://bing-spell-check2.p.rapidapi.com/spellcheck"

    querystring = {"mode":"spell","text":{word}}
    headers = {
        'X-RapidAPI-Key': 'cf81e48b98msh94ed912ed959422p1457b5jsn07de0efd6434',
        'X-RapidAPI-Host': 'bing-spell-check2.p.rapidapi.com'
    }

    response = requests.get(url, headers=headers, params=querystring)

    res = response.json()
    if(len(res)>0 ):
        if(len(res['flaggedTokens'])>0):
            suggestion = res['flaggedTokens'][0]['suggestions'][0]['suggestion']
            return suggestion
    else:
        return None


translator = GoogleTranslator(source='auto', target='en')

def remove_content_between_parentheses(word):
    # Expresión regular para eliminar el contenido entre paréntesis
    pattern = r'\(.*?\)'
    # Reemplazar el contenido entre paréntesis con una cadena vacía
    word_without_parentheses = re.sub(pattern, '', word)
    return word_without_parentheses

def extract_camel_case_words(words):
    camel_case_words = []
    for word in words:
        if re.match(r'[A-Za-z]+(?:[A-Z][a-z]+)+$', word):
            camel_case_words.append(word)
    return camel_case_words


def separate_camel_case_words(camel_case_words):
    separated_words = []
    for word in camel_case_words:
        separated_words.extend(re.findall(r'[A-Z][a-z]+|[A-Z]+(?![a-z])|[a-z]+', word))
    return separated_words

def remove_single_letter_words(words):
    return [word for word in words if  len(word) > 1]

def remove_duplicates(lista):
    lista_sin_duplicados = []
    for elemento in lista:
        if elemento not in lista_sin_duplicados:
            lista_sin_duplicados.append(elemento)
    return lista_sin_duplicados

namesList = []
coordList = []
keys_with_duplicates = []
final_properties = []
synonymous = []
properties = []
url = 'https://geoserveis.ide.cat/servei/catalunya/inspire/ogc/features/collections/inspire:PS.ProtectedSite.Natura2000/items?f=json&limit=2'

try:
    response = requests.get(url)
    data = response.json()
    feature = data['features'][0]
    properties = feature['properties']

    # Insertamos los nombres de cada feature
    for feature in data['features']:
        nameFeature = feature['id']
        namesList.insert(len(namesList),str(nameFeature))
    # Insertamos las coordenadas de cada feature
    for feature in data ['features']:
        coordenadas = feature['geometry']['coordinates']
        for coordenada in coordenadas:
            coordList.insert(len(coordList),coordenada)




    # Creamos dos arrays para almacenar las claves y los valores iniciales
    keys = []
    values = []
    for key, value in properties.items():
        keys.append(key)
        values.append(value)




    # Separamos las palabras unidas por "_", "-", y "."
    separated_keys = []
    for value in keys:
        separated_words = []
        for word in str(value).split('_'):
            separated_words.extend(word.split('-'))
        for word in separated_words:
            separated_keys.extend(word.split('.'))


    # Borramos palabras dentro de parentesis, palabras de una sola letra, palabras camelCase
    keys_filtered1 = [remove_content_between_parentheses(word) for word in separated_keys]
    keys_filtered2 = remove_single_letter_words( keys_filtered1)
    camel_case_words = extract_camel_case_words(keys_filtered2)
    # Separamos las palabras camelCase
    separated_words = separate_camel_case_words(camel_case_words)

    keys_delete =  [x for x in keys_filtered2 if x not in camel_case_words]

    for value in separated_words:
        keys_with_duplicates.insert(len(keys_with_duplicates), value)

    for value in keys_delete:
       keys_with_duplicates.insert(len(keys_with_duplicates), value)

    # Borramos las palabras duplicadas
    keys_without_duplicates = remove_duplicates(keys_with_duplicates)

    properties = []
    # Insertamos las sugerencias
    for value in keys_without_duplicates:
        resul = make_suggestion(value)
        sleep(1)
        if resul is not None:
            properties.insert(len(properties),resul)
        else:
            properties.insert(len(properties),value)

    # Realizamos la traduccion al ingles e insertamos
    for value in properties:
        translation = translator.translate(value)
        if translation is not None and " "  in translation :
            traduccion_sin_espacios = translation.replace(" ", "")
            final_properties.append(traduccion_sin_espacios)
        else:
            final_properties.append(translation)

    # Insertamos los sinonimos
    for value in final_properties:
        synonym = make_synonimous(value, "EN", "EN")
        if synonym is not None and " "  in synonym :
            sinonimo_sin_espacios = synonym.replace(" ", "")
            synonymous.append(sinonimo_sin_espacios)
        else:
            synonymous.append(synonym)


    print("Propiedades originales: ", keys)
    print("Propiedades preprocesadas: ", final_properties)
    print("Sinonimos de las propiedades: ",synonymous)



except requests.exceptions.RequestException as e:
    print('Error al hacer la solicitud:', e)

## Código para preprocesar los valores de las propiedades de los features:

In [ ]:


def remove_content_between_parentheses(word):
    # Expresión regular para eliminar el contenido entre paréntesis
    pattern = r'\(.*?\)'
    # Reemplazar el contenido entre paréntesis con una cadena vacía
    word_without_parentheses = re.sub(pattern, '', word)
    return word_without_parentheses

def extract_camel_case_words(words):
    camel_case_words = []
    for word in words:
        if re.match(r'[A-Za-z]+(?:[A-Z][a-z]+)+$', word):
            camel_case_words.append(word)
    return camel_case_words


def separate_camel_case_words(camel_case_words):
    separated_words = []
    for word in camel_case_words:
        separated_words.extend(re.findall(r'[A-Z][a-z]+|[A-Z]+(?![a-z])|[a-z]+', word))
    return separated_words

def remove_single_letter_words(words):
    return [word for word in words if  len(word) > 2]

valuesList = []
valuesInicList = []
final_values = []
url = 'https://geoserveis.ide.cat/servei/catalunya/inspire/ogc/features/collections/inspire:PS.ProtectedSite.Natura2000/items?f=json&limit=2'


try:
    for feature in data['features']:
        properties = feature['properties']
        response = requests.get(url)
        data = response.json()


        # Creamos dos arrays para almacenar las claves y los valores iniciales
        keys = []
        values = []

        for key, value in properties.items():
            keys.append(key)
            values.append(value)

        # Insertamos la lista de propiedades iniciales de cada feature
        valuesInicList.insert(len(valuesInicList),values)


        # Creamos un array para almacenar los valores filtrados
        values_filtered = []

        # Filtramos y agregamos solo los valores deseados al array values_filtered
        for value in values:
            if (
                value not in [True,False, None]
                and not isinstance(value, (int, float))
                and not re.match(r'\d{4}-\d{2}-\d{2}$', str(value))
                and not re.search(r'\d', str(value))
                and not re.search("http", str(value))
                and not re.search("https", str(value))
            ):

                values_filtered.append(value)


        # Separamos las palabras unidas por "_", "-", y "."
        separated_values = []
        for value in values_filtered:
            separated_words = []
            for word in str(value).split('_'):
                separated_words.extend(word.split('-'))
            for word in separated_words:
                separated_values.extend(word.split('.'))


        # Borramos palabras dentro de parentesis, palabras de una sola letra, palabras camelCase
        values_filtered1 = [remove_content_between_parentheses(word) for word in separated_values]
        values_filtered2 = remove_single_letter_words( values_filtered1)
        camel_case_words = extract_camel_case_words(values_filtered2)
        #Separamos las palabras camelCase
        separated_words = separate_camel_case_words(camel_case_words)
        values_delete =  [x for x in values_filtered2 if x not in camel_case_words]

        for value in separated_words:
            final_values.insert(len(final_values), value)

        for value in values_delete:
            if '\r' in value:
                indice = value.find('\r')
                value = value[:indice] + value[indice+1:]
                final_values.insert(len(final_values), value)
            elif value.startswith(" "):
                value = value.lstrip()
                final_values.insert(len(final_values), value)
            else:
                final_values.insert(len(final_values), value)

        # Insertamos los valores filtrados
        valuesList.insert(len(valuesList),final_values)


    print("Valores iniciales: ",valuesInicList)
    print("Valores preprocesados: ",valuesList)



except requests.exceptions.RequestException as e:
    print('Error al hacer la solicitud:', e)


In [88]:
def buscar_palabra_en_lista(palabra, lista_palabras):
    for indice, palabra_lista in enumerate(lista_palabras):
        if (
                palabra_lista not in [True,False, None]
                and not isinstance(palabra_lista, (int, float))
                and not re.match(r'\d{4}-\d{2}-\d{2}$', str(palabra_lista))
                and not re.search(r'\d', str(palabra_lista))
            ):

            if palabra in lista_palabras:
                return  lista_palabras.index(palabra)
            elif palabra_lista.find(palabra) != -1:
                return indice

    return -1


## Codigo para buscar coordenadas en Wikidata

In [ ]:

def get_coordinates_wikidata(instance):

    # Establecer el endpoint SPARQL de Wikidata
    endpoint = "https://query.wikidata.org/sparql"

    # Construir la consulta SPARQL con el parámetro de la instancia y la propiedad de las coordenadas
    query = """ SELECT ?o WHERE """ + """{ <""" + instance + """> wdt:P625 ?o }"""

    # Establecer la configuración del SPARQLWrapper
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Ejecutar la consulta y obtener los resultados
    results = sparql.query().convert()

    # Extraer las coordenadas de los resultados
    coordinates = []
    for result in results["results"]["bindings"]:
        coordinates.append(result["o"]["value"])

    # Procesar las coordenadas y extraer la longitud y latitud
    parsed_coordinates = []
    for coordinate in coordinates:
        parts = coordinate.split("(")[1].split(")")[0].split()
        longitude = float(parts[0])
        latitude = float(parts[1])
        parsed_coordinates.append((longitude, latitude))
    if len(parsed_coordinates)>0:
        return parsed_coordinates[0]
    else:
        return parsed_coordinates



## Codigo para buscar coordenadas en Dbpedia

In [89]:


def get_coordinates_dbpedia(instance):

    # Establecer el endpoint SPARQL de DBpedia
    endpoint = "http://dbpedia.org/sparql"

    # Construir la consulta SPARQL con la instancia y las propiedades de latitud y longitud
    query = """
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT ?lat ?long
    WHERE {
      <""" + instance + """> geo:lat ?lat ;
                   geo:long ?long .
    }
    """
    # Establecer la configuración del SPARQLWrapper
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Ejecutar la consulta y obtener los resultados
    results = sparql.query().convert()

    # Extraer las coordenadas de los resultados
    coordinates = []
    for result in results["results"]["bindings"]:
        latitude = float(result["lat"]["value"])
        longitude = float(result["long"]["value"])
        coordinates.append((latitude, longitude))

    if len(coordinates)>0:
        return coordinates[0]
    else:
        return coordinates


## Codigo para buscar coordenadas en LinkedGeoData

In [57]:

def get_coordinates_lgd(instance):

    # Establecer el endpoint SPARQL de GeoLinkedData
    endpoint = "http://linkedgeodata.org/sparql"

    # Construir la consulta SPARQL con la instancia y las propiedades de latitud y longitud
    query = """
    PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
    SELECT DISTINCT ?lat ?long
    WHERE { <""" +  instance + """> geo:lat ?lat .
      <""" + instance + """> geo:long ?long .
    }
    """

    # Establecer la configuración del SPARQLWrapper
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Ejecutar la consulta y obtener los resultados
    results = sparql.query().convert()

    # Extraer las coordenadas de los resultados
    coordinates = []
    for result in results["results"]["bindings"]:
        latitude = float(result["lat"]["value"])
        longitude = float(result["long"]["value"])
        coordinates.append((latitude, longitude))

    if len(coordinates)>0:
        return coordinates[0]
    else:
        return coordinates



## Codigo para inicializar el grafo de conocimiento

In [138]:
# Crear un objeto Graph
g = Graph()
# Definir el namespace de la ontologia
my_ns = Namespace("https://geo.linkeddata.es/def/ogcapi-feature#")



## Codigo para buscar las instancias de los valores en Wikidata

In [104]:

# Establecer el punto de enlace de SPARQL de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

for list in valuesList:
    for value in list:

        # Definimos la consulta SPARQL
        query = """
        SELECT  ?WikiC
                      WHERE {
                        ?WikiC rdfs:label  """ + '"' + value + """"@en.
                      }
        """

        # Establecemos la consulta SPARQL y el formato de salida
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Ejecutamos la consulta y obtener los resultados
        res = sparql.query().convert()



        if(len(res['results']['bindings'])>0):

            # Obtener todos los valores de la clave 'value'
            resul = [item['WikiC']['value'] for item in res['results']['bindings']]

            # Procesamos las instancias obtenidas
            for val in resul:
                # Buscamos las coordenada de la instancia obtenida
                coordenadas = get_coordinates_wikidata(val)
                if len(coordenadas)>0:
                    point = Point(coordenadas)
                    coordinates = coordList[valuesList.index(list)]
                    polygon = Polygon(coordinates)
                    # Las coordenadas de la instancia estan dentro de las coordenadas del feature
                    if point.within(polygon):
                        s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
                        p = URIRef("http://www.opengis.net/ont/geosparql#sfContains")
                        o = URIRef(val)
                        g.add((s, p, o))
                        listInic = valuesInicList[valuesList.index(list)]
                        index = buscar_palabra_en_lista(value,listInic)
                        if index!=-1:
                            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#" + keys[index])
                            o = URIRef(val)
                            g.add((s, p, o))
                            if isinstance(value, str) and " " in value:
                                value = value.replace(" ", "")
                            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + value)
                            p = URIRef("http://www.opengis.net/ont/geosparql#sfContains")
                            o = URIRef(val)
                            g.add((s, p, o))

                # La instancia no tiene coordenadas
                else:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
                    p = URIRef("http://www.w3.org/2004/02/skos/core#related")
                    if isinstance(value, str) and " " in value:
                        value = value.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/"+ value)
                    g.add((s, p, o))

        # No se han encontrado instancias en Wikidata con ese valor
        else:
            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
            if isinstance(value, str) and " " in value:
                value = value.replace(" ", "")
            o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/"+ value)
            g.add((s, p, o))


    file_path = "archivo_wikidata_values.rdf"
    g.serialize(file_path, format="ttl")




## Codigo para buscar las instancias de las propiedades en Wikidata

In [ ]:

# Establecer el punto de enlace de SPARQL de Wikidata
sparql = SPARQLWrapper("http://query.wikidata.org/sparql")

for name in namesList:
    for property in final_properties:
        # Definimos la consulta SPARQL
        query = """
        SELECT  ?WikiP
                      WHERE {
                        ?WikiP rdfs:label  """ + '"' + property + """"@en.
                      }
        """


        # Establecemos la consulta SPARQL y el formato de salida
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Ejecutamos la consulta y obtener los resultados
        res = sparql.query().convert()


        if(len(res['results']['bindings'])>0):
            # Obtener todos los valores de la clave 'value'
            resul = [item['WikiP']['value'] for item in res['results']['bindings']]
            # Procesamos las instancias obtenidas
            for val in resul:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                palabra = keys_without_duplicates[final_properties.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    p = Literal(val)
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

        # No se han encontrado instancias en Wikidata con ese valor
        else:
            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
            palabra = keys_without_duplicates[final_properties.index(property)]
            index = buscar_palabra_en_lista(palabra,keys)
            if index!=-1:
                indVal = namesList.index(name)
                listaVal = valuesInicList[indVal]
                palabra = listaVal[index]
                if isinstance(palabra, str) and " " in palabra:
                    palabra = palabra.replace(" ", "")
                o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                g.add((s, p, o))


    file_path = "archivo_wikidata_prop.rdf"
    g.serialize(file_path, format="ttl")


## Codigo para buscar las instancias de los sinonimos en Wikidata

In [112]:

# Establecer el punto de enlace de SPARQL de Wikidata
sparql = SPARQLWrapper("http://query.wikidata.org/sparql")

for name in namesList:
    for property in synonymous:
        if property is not None:

            # Definimos la consulta SPARQL
            query = """
            SELECT  ?WikiP
                          WHERE {
                            ?WikiP rdfs:label  """ + '"' + property + """"@en.
                          }
            """

            # Establecemos la consulta SPARQL y el formato de salida
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # Ejecutamos la consulta y obtener los resultados
            res = sparql.query().convert()
            if (len(res['results']['bindings']) > 0):
                # Obtener todos los valores de la clave 'value'
                resul = [item['WikiP']['value'] for item in res['results']['bindings']]

                # Procesamos las instancias obtenidas
                for val in resul:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                    palabra = keys_without_duplicates[synonymous.index(property)]
                    index = buscar_palabra_en_lista(palabra,keys)
                    if index!=-1:
                        p = Literal(val)
                        indVal = namesList.index(name)
                        listaVal = valuesInicList[indVal]
                        palabra = listaVal[index]
                        if isinstance(palabra, str) and " " in palabra:
                            palabra = palabra.replace(" ", "")
                        o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                        g.add((s, p, o))

            # No se han encontrado instancias en Wikidata con ese valor
            else:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
                palabra = keys_without_duplicates[synonymous.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

    file_path = "archivo_wikidata_prop_syn.rdf"
    g.serialize(file_path, format="ttl")



## Codigo para buscar instancias de las propiedades y sinonimos en Wikidata

In [136]:

def execute_sparql_query(query):
    # Establecemos la consulta SPARQL y el formato de salida
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Ejecutamos la consulta y obtener los resultados
    return sparql.query().convert()



# Establecemos el punto de enlace de SPARQL de Wikidata
sparql = SPARQLWrapper("http://query.wikidata.org/sparql")

for name in namesList:
    for pos, property in enumerate(final_properties):
        # Definimos la consulta SPARQL
        query = """
        SELECT  DISTINCT ?WikiP
                          WHERE {
                            ?WikiP rdfs:label  """ + '"' + property + """"@en.
                          }
            """

        # Ejecutamos la consulta y obtenemos los resultados
        res = execute_sparql_query(query)
        if(len(res['results']['bindings'])>0):
            # Obtener todos los valores de la clave 'value'
            resul = [item['WikiP']['value'] for item in res['results']['bindings']]
            # Procesamos las instancias obtenidas
            for val in resul:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                palabra = keys_without_duplicates[final_properties.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    p = Literal(val)
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

        #Buscamos con el sinonimo
        else:
            property = synonymous[pos]
            if property is not None:
                # Definimos la consulta SPARQL con sinonimo
                querySyn = """
                  SELECT  ?WikiP
                          WHERE {
                            ?WikiP rdfs:label  """ + '"' + property + """"@en.
                          }
            """
                # Ejecutar la consulta y obtener los resultados
                res = execute_sparql_query(querySyn)
                if(len(res['results']['bindings'])>0):
                    # Obtener todos los valores de la clave 'value'
                    resul = [item['WikiP']['value'] for item in res['results']['bindings']]
                    # Procesamos las instancias obtenidas
                    for val in resul:
                        s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                        palabra = keys_without_duplicates[synonymous.index(property)]
                        index = buscar_palabra_en_lista(palabra,keys)
                        if index!=-1:
                            p = Literal(val)
                            indVal = namesList.index(name)
                            listaVal = valuesInicList[indVal]
                            palabra = listaVal[index]
                            if isinstance(palabra, str) and " " in palabra:
                                palabra = palabra.replace(" ", "")
                            o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                            g.add((s, p, o))

                #  No se han encontrado instancias en Wikidata con ese valor
                else:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                    p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
                    palabra = keys_without_duplicates[synonymous.index(property)]
                    index = buscar_palabra_en_lista(palabra,keys)
                    if index!=-1:
                        indVal = namesList.index(name)
                        listaVal = valuesInicList[indVal]
                        palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

            # El sinonimo es None
            else:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
                palabra = keys_without_duplicates[synonymous.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                if isinstance(palabra, str) and " " in palabra:
                    palabra = palabra.replace(" ", "")
                o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                g.add((s, p, o))

    file_path = "archivo_wikidata_prop_and_syn.rdf"
    g.serialize(file_path, format="ttl")

## Codigo para buscar las instancias de los valores en Dbpedia

In [124]:

for list in valuesList:
    for value in list:

        # Establecemos el punto de enlace de SPARQL de DBpedia
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")

        # Definimos la consulta SPARQL
        query = """
        SELECT  ?DbC
                      WHERE {
                        ?DbC rdfs:label  """ + '"' + value + """"@en.
                      }
        """

        # Establecemos la consulta SPARQL y el formato de salida
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Ejecutamos la consulta y obtener los resultados
        res = sparql.query().convert()
        if(len(res['results']['bindings'])>0):
            # Obtener todos los valores de la clave 'value'
            resul = [item['DbC']['value'] for item in res['results']['bindings']]
            # Procesamos las instancias obtenidas
            for val in resul:
                #Buscamos las coordenadas de la instancia obtenida
                coordenadas = get_coordinates_dbpedia(val)
                if len(coordenadas)>0:
                    point = Point(coordenadas)
                    coordinates = coordList[valuesList.index(list)]
                    polygon = Polygon(coordinates)
                    # Las coordenadas de la instancia estan dentro de las coordenadas del feature
                    if point.within(polygon):
                        s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
                        p = URIRef("http://www.opengis.net/ont/geosparql#sfContains")
                        o = URIRef(val)
                        g.add((s, p, o))
                        listInic = valuesInicList[valuesList.index(list)]
                        index = buscar_palabra_en_lista(value,listInic)
                        if index!=-1:
                            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#" + keys[index])
                            o = URIRef(val)
                            g.add((s, p, o))
                            if isinstance(value, str) and " " in value:
                                value = value.replace(" ", "")
                            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + value)
                            p = URIRef("http://www.opengis.net/ont/geosparql#sfContains")
                            o = URIRef(val)
                            g.add((s, p, o))

                # La instancia no tiene coordenadas
                else:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
                    p = URIRef("http://www.w3.org/2004/02/skos/core#related")
                    if isinstance(value, str) and " " in value:
                        value = value.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/"+ value)
                    g.add((s, p, o))

        # No se han encontrado instancias en Dbpedia con ese valor
        else:
            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
            if isinstance(value, str) and " " in value:
                value = value.replace(" ", "")
            o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/"+ value)
            g.add((s, p, o))

    file_path = "archivo_dbpedia_values.rdf"
    g.serialize(file_path, format="ttl")


## Codigo para buscar las instancias de las propiedades en Dbpedia

In [125]:

for name in namesList:
    for property in final_properties:

        # Establecemos el punto de enlace de SPARQL de DBpedia
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")

        # Definimos la consulta SPARQL
        query = """
        SELECT  ?DbP
                      WHERE {
                        ?DbP rdfs:label  """ + '"' + property + """"@en.
                      }
        """

         # Establecemos la consulta SPARQL y el formato de salida
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Ejecutamos la consulta y obtener los resultados
        res = sparql.query().convert()

        if(len(res['results']['bindings'])>0):
            # Obtener todos los valores de la clave 'value'
            resul = [item['DbP']['value'] for item in res['results']['bindings']]
            # Procesamos las instancias obtenidas
            for val in resul:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                    palabra = keys_without_duplicates[final_properties.index(property)]
                    index = buscar_palabra_en_lista(palabra,keys)
                    if index!=-1:
                        p = Literal(val)
                        indVal = namesList.index(name)
                        listaVal = valuesInicList[indVal]
                        palabra = listaVal[index]
                        if isinstance(palabra, str) and " " in palabra:
                            palabra = palabra.replace(" ", "")
                        o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                        g.add((s, p, o))

        # No se han encontrado instancias en Dbpedia con ese valor
        else:
            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
            palabra = keys_without_duplicates[final_properties.index(property)]
            index = buscar_palabra_en_lista(palabra,keys)
            if index!=-1:
                indVal = namesList.index(name)
                listaVal = valuesInicList[indVal]
                palabra = listaVal[index]
                if isinstance(palabra, str) and " " in palabra:
                    palabra = palabra.replace(" ", "")
                o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                g.add((s, p, o))

    file_path = "archivo_dbpedia_prop.rdf"
    g.serialize(file_path, format="ttl")


## Codigo para buscar las instancias de los sinonimos en Dbpedia

In [127]:

# Establecemos el punto de enlace de SPARQL de DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

for name in namesList:
    for property in synonymous:
         if property is not None:
            # Definimos la consulta SPARQL
            query = """
            SELECT  ?DbP
                          WHERE {
                            ?DbP rdfs:label  """ + '"' + property + """"@en.
                          }
            """

             # Establecemos la consulta SPARQL y el formato de salida
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)

            # Ejecutamos la consulta y obtener los resultados
            res = sparql.query().convert()
            if(len(res['results']['bindings'])>0):
                # Obtener todos los valores de la clave 'value'
                resul = [item['DbP']['value'] for item in res['results']['bindings']]
                # Procesamos las instancias obtenidas
                for val in resul:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                    palabra = keys_without_duplicates[synonymous.index(property)]
                    index = buscar_palabra_en_lista(palabra,keys)
                    if index!=-1:
                        p = Literal(val)
                        indVal = namesList.index(name)
                        listaVal = valuesInicList[indVal]
                        palabra = listaVal[index]
                        if isinstance(palabra, str) and " " in palabra:
                            palabra = palabra.replace(" ", "")
                        o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                        g.add((s, p, o))

            # No se han encontrado instancias en Dbpedia con ese valor
            else:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
                palabra = keys_without_duplicates[synonymous.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

    file_path = "archivo_dbpedia_prop_syn.rdf"
    g.serialize(file_path, format="ttl")

## Codigo para buscar instancias de las propiedades y sinonimos en Dbpedia

In [ ]:


def execute_sparql_query(query):
    # Establecer la consulta SPARQL y el formato de salida
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Ejecutar la consulta y obtener los resultados
    return sparql.query().convert()


# Establecemos el punto de enlace de SPARQL de DBpedia
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

for name in namesList:
    for pos, property in enumerate(final_properties):

        # Definimos la consulta SPARQL
        query = """
         SELECT DISTINCT ?DbP
                          WHERE {
                            ?DbP rdfs:label  """ + '"' + property + """"@en.
                          }
            """



        # Ejecutamos la consulta y obtener los resultados
        res = execute_sparql_query(query)
        if(len(res['results']['bindings'])>0):
            # Obtener todos los valores de la clave 'value'
            resul = [item['DbP']['value'] for item in res['results']['bindings']]
            # Procesamos las instancias obtenidas
            for val in resul:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                palabra = keys_without_duplicates[final_properties.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    p = Literal(val)
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

        #Buscamos con el sinonimo
        else:
            property = synonymous[pos]
            if property is not None:
                # Definir la consulta SPARQL con sinonimo
                querySyn = """
                  SELECT  ?DbP
                          WHERE {
                            ?DbP rdfs:label  """ + '"' + property + """"@en.
                          }
                  """

                 # Ejecutamos la consulta y obtener los resultados
                res = execute_sparql_query(querySyn)
                if(len(res['results']['bindings'])>0):
                    # Obtener todos los valores de la clave 'value'
                    resul = [item['DbP']['value'] for item in res['results']['bindings']]
                    # Procesamos las instancias obtenidas
                    for val in resul:
                        s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                        palabra = keys_without_duplicates[synonymous.index(property)]
                        index = buscar_palabra_en_lista(palabra,keys)
                        if index!=-1:
                            p = Literal(val)
                            indVal = namesList.index(name)
                            listaVal = valuesInicList[indVal]
                            palabra = listaVal[index]
                            if isinstance(palabra, str) and " " in palabra:
                                palabra = palabra.replace(" ", "")
                            o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                            g.add((s, p, o))

                # No se han encontrado instancias en Dbpedia con ese valor
                else:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                    p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
                    palabra = keys_without_duplicates[synonymous.index(property)]
                    index = buscar_palabra_en_lista(palabra,keys)
                    if index!=-1:
                        indVal = namesList.index(name)
                        listaVal = valuesInicList[indVal]
                        palabra = listaVal[index]
                        if isinstance(palabra, str) and " " in palabra:
                            palabra = palabra.replace(" ", "")
                        o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                        g.add((s, p, o))
            # El sinonimo es None
            else:
                s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + name)
                p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
                palabra = keys_without_duplicates[synonymous.index(property)]
                index = buscar_palabra_en_lista(palabra,keys)
                if index!=-1:
                    indVal = namesList.index(name)
                    listaVal = valuesInicList[indVal]
                    palabra = listaVal[index]
                    if isinstance(palabra, str) and " " in palabra:
                        palabra = palabra.replace(" ", "")
                    o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + str(palabra))
                    g.add((s, p, o))

    file_path = "archivo_dbpedia_prop_and_syn.rdf"
    g.serialize(file_path, format="ttl")

## Codigo para buscar las instancias de los valores en LinkedGeoData

In [139]:


# Establecer el punto de enlace de SPARQL de LinkedGeoData
sparql = SPARQLWrapper("http://linkedgeodata.org/sparql")

for list in valuesList:
    for value in list:

        # Definimos la consulta SPARQL
        query = """
        SELECT  ?LgdC
                      WHERE {
                        ?LgdC rdfs:label  """ + '"' + value + """".
                      }
        """

        # Establecemos la consulta SPARQL y el formato de salida
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        # Ejecutamos la consulta y obtener los resultados
        res = sparql.query().convert()
        if(len(res['results']['bindings'])>0):
            # Obtener todos los valores de la clave 'value'
            resul = [item['LgdC']['value'] for item in res['results']['bindings']]
            # Procesamos las instancias obtenidas
            for val in resul:
                # Buscamos las coordenadas de las instancias
                coordenadas = get_coordinates_lgd(val)
                if len(coordenadas)>0:
                    point = Point(coordenadas)
                    coordinates = coordList[valuesList.index(list)]
                    polygon = Polygon(coordinates)
                    # Las coordenadas de las instancia estan dentro de las coordenadas de la feature
                    if point.within(polygon):
                        s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
                        p = URIRef("http://www.opengis.net/ont/geosparql#sfContains")
                        o = URIRef(val)
                        g.add((s, p, o))
                        listInic = valuesInicList[valuesList.index(list)]
                        index = buscar_palabra_en_lista(value,listInic)
                        if index!=-1:
                            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#" + keys[index])
                            o = URIRef(val)
                            g.add((s, p, o))
                            if isinstance(value, str) and " " in value:
                                value = value.replace(" ", "")
                            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + value)
                            p = URIRef("http://www.opengis.net/ont/geosparql#sfContains")
                            o = URIRef(val)
                            g.add((s, p, o))

                # La instancia no tiene coordenadas
                else:
                    s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
                    valList = valuesInicList[valuesList.index(list)]
                    index = buscar_palabra_en_lista(value,valList)
                    if index!=-1:
                        p = URIRef("http://www.w3.org/2004/02/skos/core#related")
                        if isinstance(value, str) and " " in value:
                            value = value.replace(" ", "")
                        o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/"+ value)
                        g.add((s, p, o))

        # No se han encontrado instancias en Wikidata con ese valor
        else:
            s = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/" + namesList[valuesList.index(list)])
            p = URIRef("https://geo.linkeddata.es/def/ogcapi-feature#hasApiProperty")
            if isinstance(value, str) and " " in value:
                value = value.replace(" ", "")
            o = URIRef("https://geo.linkeddata.es/geoserveis.ide.cat/feature/"+ value)
            g.add((s, p, o))

    file_path = "archivo_geolinkeddata_values.rdf"
    g.serialize(file_path, format="ttl")
